In [1]:
import os
import geopandas as gpd
import json
import rasterio
from rasterio.mask import mask
from shapely.geometry import mapping
from unidecode import unidecode
from rasterio.warp import calculate_default_transform, reproject, Resampling
import richdem


In [2]:
def clip_reproject_raster(input_raster_path, region_name_clean, gdf, landcover_elevation, target_crs, resampling_method, output_dir):
    """
        Reads a TIFF raster, clips it to the extent of a GeoPandas DataFrame, reprojects it to a given CRS considerung the set resampling method,
        and saves the clipped raster in a specified output folder.
    
        :param input_raster_path: Path to the input TIFF raster file.
        :param region_name_clean: in main script cleaned region name
        :param gdf: The GeoPandas DataFrame to use for clipping the raster.
        :param landcover_elevation: string with "landcover" or "elevation". 
        :param target_crs: The target CRS to reproject the raster to (e.g., 'EPSG:3035').
        :param resampling_method: resampling method to be used (string)
        :param output_dir: output directory (defined in main script)
        """
    
    resampling_options = {
        'nearest': Resampling.nearest,
        'bilinear': Resampling.bilinear,
        'cubic': Resampling.cubic
    }

    with rasterio.open(input_raster_path) as src:
        # Mask the raster using the vector file's geometry
        out_image, out_transform = mask(src, gdf.geometry.apply(mapping), crop=True)
        # Copy the metadata from the source raster
        out_meta = src.meta.copy()
        # Update the metadata for the clipped raster
        out_meta.update({
            'height': out_image.shape[1],
            'width': out_image.shape[2],
            'transform': out_transform
        })

        ori_raster_crs = str(src.crs)
        ori_raster_crs = ori_raster_crs.replace(":", "")
        print(f'original raster CRS: {src.crs}')
        # Save the clipped raster as a new GeoTIFF file
        with rasterio.open(os.path.join(output_dir, f'{landcover_elevation}_{region_name_clean}_{ori_raster_crs}.tif'), 'w', **out_meta) as dest:
            dest.write(out_image)

    # reproject landcover raster to local UTM CRS
    with rasterio.open(os.path.join(output_dir, f'{landcover_elevation}_{region_name_clean}_{ori_raster_crs}.tif')) as src:
        # Calculate the transformation and dimensions for the target CRS
        transform, width, height = calculate_default_transform(
            src.crs, target_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': target_crs,
            'transform': transform, 
            'width': width,
            'height': height,
            'dtype': rasterio.int16
        })

        # Create the output file path
        output_path = os.path.join(output_dir, f'{landcover_elevation}_{region_name_clean}_EPSG{target_crs}.tif')


        # Reproject and save the raster
        with rasterio.open(output_path, 'w', **kwargs, compress='DEFLATE') as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=target_crs,
                    resampling=resampling_options[resampling_method])
                
            print(f'reprojected raster CRS: {dst.crs}') 




def co_register(infile, match, resampling_method, outfile): #source: https://pygis.io/docs/e_raster_resample.html
    """Reproject a file to match the shape and projection of existing raster. (co-registration)
    
    Parameters
    ----------
    infile : (string) path to input file to reproject
    match : (string) path to raster with desired shape and projection 
    outfile : (string) path to output file tif
    """
    resampling_options = {
        'nearest': Resampling.nearest,
        'bilinear': Resampling.bilinear,
        'cubic': Resampling.cubic
    }

    # open input
    with rasterio.open(infile) as src:
        src_transform = src.transform
        
        # open input to match
        with rasterio.open(match) as match:
            dst_crs = match.crs
            
            # calculate the output transform matrix
            dst_transform, dst_width, dst_height = calculate_default_transform(
                src.crs,     # input CRS
                dst_crs,     # output CRS
                match.width,   # input width
                match.height,  # input height 
                *match.bounds,  # unpacks input outer boundaries (left, bottom, right, top)
            )

        # set properties for output
        dst_profile = src.profile.copy()
        
        #dst_kwargs.update({"crs": dst_crs,
        #                   "transform": dst_transform,
        #                   "width": dst_width,
        #                   "height": dst_height,
        #                   "nodata": -9999,
        #                   "dtype": rasterio.int16})
        
        dst_profile.update(
            crs=dst_crs,
            transform=dst_transform,
            width=dst_width,
            height=dst_height,
            dtype=rasterio.int16,
            count=1,
            nodata=-9999,
            compress='DEFLATE') 
        

        print("Coregistered to shape:", dst_height,dst_width,'\n Affine',dst_transform)
        # open output
        with rasterio.open(outfile, "w", **dst_profile) as dst:
            # iterate through bands and write using reproject function
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=dst_transform,
                    dst_crs=dst_crs,
                    resampling=resampling_options[resampling_method])
                
def save_richdem_file(richdem_file, base_dem_FilePath, outFilePath):
    with rasterio.open(base_dem_FilePath) as src:
        file_profile = src.profile
    # For the new file's profile, we start with the profile of the source
    profile = file_profile

    profile.update(
        dtype=rasterio.int16,
        count=1,
        compress='DEFLATE',
        nodata=richdem_file.no_data) 
    #save raster file
    with rasterio.open(outFilePath, 'w', **profile) as dst:
        dst.write(richdem_file, 1)  #richdem_file.astype(rasterio.int16)

In [ ]:
dirname = os.getcwd()
data_path = os.path.join(dirname, '..', 'Raw_Spatial_Data')
#landcoverRasterPath = os.path.join(data_path, "PROBAV_LC100_global_v3.0.1_2019-nrt_Discrete-Classification-map_EPSG-4326.tif")
demRasterPath = os.path.join(data_path, 'DEM','gebco_cutout.tif')


region_name_clean = 'Elbe-Elster'
regionDataPath = os.path.join('..', 'data', region_name_clean)
EPSG = 32633
region = gpd.read_file(os.path.join(regionDataPath, f'{region_name_clean}_4326.geojson'))

output_dir = dirname


matchPath=os.path.join(dirname, f'landcover_{region_name_clean}_EPSG{EPSG}.tif')


In [14]:
dem_localCRS_Path=os.path.join(f'DEM_{region_name_clean}_EPSG{EPSG}.tif')
matchPath=os.path.join(output_dir, f'landcover_{region_name_clean}_EPSG{EPSG}.tif')


dem_file = richdem.LoadGDAL(dem_localCRS_Path)
slope = richdem.TerrainAttribute(dem_file, attrib='slope_degrees')
slopeFilePath = os.path.join(f'slope_{region_name_clean}_EPSG{EPSG}.tif')
save_richdem_file(slope, dem_localCRS_Path, slopeFilePath)
co_register(slopeFilePath, matchPath, 'nearest', os.path.join(f'slope_{region_name_clean}_EPSG{EPSG}_resampled.tif'))
    

Coregistered to shape: 8342 8401 
 Affine | 7.06, 0.00, 364328.69|
| 0.00,-7.06, 5750165.31|
| 0.00, 0.00, 1.00|


In [4]:
clip_reproject_raster(demRasterPath, region_name_clean, region, 'DEM', EPSG, 'nearest', output_dir)


original raster CRS: EPSG:4326
reprojected raster CRS: EPSG:32633


In [11]:
dem=os.path.join(output_dir, f'DEM_{region_name_clean}_EPSG{EPSG}.tif')

dem_file = richdem.LoadGDAL(f'DEM_{region_name_clean}_EPSG{EPSG}.tif')
slope = richdem.TerrainAttribute(dem_file, attrib='slope_degrees')
save_richdem_file(slope, dem, os.path.join(output_dir, f'slope_{region_name_clean}_EPSG{EPSG}.tif'))


In [9]:
type(slope)

richdem.rdarray

In [ ]:
slope_orig=os.path.join(dirname, f'DEM_{region_name_clean}_EPSG{EPSG}.tif')
match=os.path.join(dirname, f'landcover_{region_name_clean}_EPSG{EPSG}.tif')
slope_resampled=os.path.join(dirname, f'slope_{region_name_clean}_EPSG{EPSG}_resampled.tif')

co_register(slope, match, 'nearest', slope_resampled)

TypeError: invalid path or file: rdarray([[-9999., -9999., -9999., ..., -9999., -9999., -9999.],
         [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
         [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
         ...,
         [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
         [-9999., -9999., -9999., ..., -9999., -9999., -9999.],
         [-9999., -9999., -9999., ..., -9999., -9999., -9999.]],
        dtype=float32)

In [ ]:
#reproject and match resolution of DEM to landcover data (co-registration)
dem_orig=os.path.join(dirname, f'DEM_{region_name_clean}_EPSG{EPSG}.tif')
match=os.path.join(dirname, f'landcover_{region_name_clean}_EPSG{EPSG}.tif')
dem_resampled=os.path.join(dirname, f'DEM_{region_name_clean}_EPSG{EPSG}_resampled.tif')
co_register(dem_orig, match, 'nearest', dem_resampled)

Coregistered to shape: 8342 8401 
 Affine | 7.06, 0.00, 364328.69|
| 0.00,-7.06, 5750165.31|
| 0.00, 0.00, 1.00|
